# Registry

In [1]:
#default_exp registry

In [2]:
#export
from torch_snippets.markup import Config, AttrDict
from torch_snippets.thinc_parser.parser import registry
Config = Config
AttrDict = AttrDict
registry = registry

In [4]:
registry.create('student')
registry.create('student')
registry.create('student')